## 02 Data Cleaning and Preprocessing
**Objetivo**: Limpiar y preprocesar los datos para que estén listos para el modelado.
**Contenido**:
- Manejo de valores nulos.
- Creación de nuevas características (feature engineering).
- Conversión de tipos de datos.
- Codificación de variables categóricas.
- Normalización y estandarización de las variables.



In [2]:
#!pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 3.5 MB/s eta 0:00:00


In [3]:
## Librerías
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
import category_encoders as ce
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, RobustScaler
import os

In [4]:
## Importamos las librerias a usar
from google.colab import drive
drive.mount('/content/drive')
data_dir = '/content/drive/MyDrive/cursos-analisis-datos/data-science/proyecto/propuestas/propuesta1/ieee-fraud-detection'


Mounted at /content/drive


##  Cargamos los Dataset

- Trabajaremos exclusivamente con el dataset de entrenamiento "train_transaction" para obtener tanto los datos de entrenamiento como los de validación. De este dataset solo tomaremos el 10 % del total de los registro.

In [ ]:
select_col_transaction = 'TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9'.split(',')
select_col_identity = 'TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo'.split(',')
data_dir = '../data/raw/ieee-fraud-detection'

In [62]:
col_selection_feature_cat_orig = ['card4', 'card6', 'M2', 'M3', 'M3','M6', 'M6', 'P_emaildomain', 'V135', 'V319', 'V137', 'V321', 'V129', 'V132', 'V309', 'V136', 'V316',
'V320','isFraud']
col_selection_feature_catg_num=['V135', 'V319', 'V137', 'V321', 'V129', 'V132', 'V309', 'V136', 'V316',
'V320']

col_selection_feature_num = ['V313', 'V134', 'V318', 'V315', 'V306', 'D2', 'V317', 'V128', 'V308','V307']


### Selección de porción del Dataset de entrenamiento con Muestreo Estratificado



In [63]:
#data_dir = '../data/raw/ieee-fraud-detection'

## dataset de entrenamiento
seed = 42
# Cargar los datos
df_transaction_train = pd.read_csv(data_dir + '/train_transaction.csv', usecols=col_selection_feature_cat_orig + col_selection_feature_catg_num+col_selection_feature_num)
#df_transaction_train = pd.read_csv(data_dir + '/train_transaction.csv', usecols=select_col_transaction
#df_identity_train = pd.read_csv(data_dir + '/train_identity.csv', usecols=select_col_identity)

# Combinar los datasets
#dataset = pd.merge(df_transaction_train, df_identity_train, on='TransactionID', how='left')
dataset = df_transaction_train.copy()
# Realizar el muestreo estratificado
train_data, _ = train_test_split(dataset, stratify=dataset['isFraud'], test_size=0.9, random_state=seed)

# Mostrar la información del dataset resultante
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59054 entries, 294161 to 404259
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   isFraud        59054 non-null  int64  
 1   card4          58896 non-null  object 
 2   card6          58896 non-null  object 
 3   P_emaildomain  49586 non-null  object 
 4   D2             30944 non-null  float64
 5   M2             31817 non-null  object 
 6   M3             31817 non-null  object 
 7   M6             42075 non-null  object 
 8   V128           59023 non-null  float64
 9   V129           59023 non-null  float64
 10  V132           59023 non-null  float64
 11  V134           59023 non-null  float64
 12  V135           59023 non-null  float64
 13  V136           59023 non-null  float64
 14  V137           59023 non-null  float64
 15  V306           59053 non-null  float64
 16  V307           59053 non-null  float64
 17  V308           59053 non-null  float64
 18  V309 

In [64]:
data = train_data.copy()

## Manejo de valores nulos

In [65]:
data.dropna(inplace=True)

In [66]:
data.isnull().sum()

isFraud          0
card4            0
card6            0
P_emaildomain    0
D2               0
M2               0
M3               0
M6               0
V128             0
V129             0
V132             0
V134             0
V135             0
V136             0
V137             0
V306             0
V307             0
V308             0
V309             0
V313             0
V315             0
V316             0
V317             0
V318             0
V319             0
V320             0
V321             0
dtype: int64

In [67]:
null_percentages = (data.isnull().sum() / len(data)) * 100
print(round(null_percentages,2).sort_values(ascending=True).head(30))

isFraud          0.0
V319             0.0
V318             0.0
V317             0.0
V316             0.0
V315             0.0
V313             0.0
V309             0.0
V308             0.0
V307             0.0
V306             0.0
V137             0.0
V320             0.0
V136             0.0
V134             0.0
V132             0.0
V129             0.0
V128             0.0
M6               0.0
M3               0.0
M2               0.0
D2               0.0
P_emaildomain    0.0
card6            0.0
card4            0.0
V135             0.0
V321             0.0
dtype: float64


In [68]:
data.shape

(14819, 27)

## Conversión de datos

No es necesatio convertir los tipos de datos, debido a que son adecuado al tipo de datos que ya tienen.

In [69]:
data.dtypes

isFraud            int64
card4             object
card6             object
P_emaildomain     object
D2               float64
M2                object
M3                object
M6                object
V128             float64
V129             float64
V132             float64
V134             float64
V135             float64
V136             float64
V137             float64
V306             float64
V307             float64
V308             float64
V309             float64
V313             float64
V315             float64
V316             float64
V317             float64
V318             float64
V319             float64
V320             float64
V321             float64
dtype: object

## Separamos datos características (X) y variable objetivo (y)

In [70]:
X = data.drop('isFraud', axis=1)  # features
y = data['isFraud']  # target

### Codificación de variables categóricas

In [71]:
X.dtypes

card4             object
card6             object
P_emaildomain     object
D2               float64
M2                object
M3                object
M6                object
V128             float64
V129             float64
V132             float64
V134             float64
V135             float64
V136             float64
V137             float64
V306             float64
V307             float64
V308             float64
V309             float64
V313             float64
V315             float64
V316             float64
V317             float64
V318             float64
V319             float64
V320             float64
V321             float64
dtype: object

In [72]:
# Identificar tipos de datos
data_types = X.dtypes

# Filtrar y contar variables numéricas y categóricas
num_vars = data_types[data_types != 'object']  # Variables numéricas
cat_vars = data_types[data_types == 'object']  # Variables categóricas
num_vars_names = num_vars.index.tolist()
cat_vars_names = cat_vars.index.tolist()

In [73]:
for columna in cat_vars_names :
    print(f"Columna: {columna}, el número de variables es: {X[columna].nunique()}")

Columna: card4, el número de variables es: 3
Columna: card6, el número de variables es: 2
Columna: P_emaildomain, el número de variables es: 44
Columna: M2, el número de variables es: 2
Columna: M3, el número de variables es: 2
Columna: M6, el número de variables es: 2


In [74]:
for columna in col_selection_feature_catg_num:
    print(f"Columna: {columna}, el número de variables es: {X[columna].nunique()}")

Columna: V135, el número de variables es: 264
Columna: V319, el número de variables es: 296
Columna: V137, el número de variables es: 343
Columna: V321, el número de variables es: 396
Columna: V129, el número de variables es: 363
Columna: V132, el número de variables es: 368
Columna: V309, el número de variables es: 426
Columna: V136, el número de variables es: 506
Columna: V316, el número de variables es: 452
Columna: V320, el número de variables es: 574


In [75]:
def one_hot_encode(df, columns):
    encoder = OneHotEncoder()
    encoded = encoder.fit_transform(df[columns])
    encoded_df = pd.DataFrame(encoded.toarray(),
                              columns=encoder.get_feature_names_out(columns),
                              index=df.index)
    df = df.drop(columns, axis=1)
    return pd.concat([df, encoded_df], axis=1)

def binary_encode(df, columns):
    encoder = ce.BinaryEncoder(cols=columns)
    encoded = encoder.fit_transform(df[columns])
    df = df.drop(columns, axis=1)
    return pd.concat([df, encoded], axis=1)

# Codificación One-Hot para ProductCD, card4 y card6
X = one_hot_encode(X, [ 'card4', 'card6','M2', 'M3','M6'])

# Codificación BinaryEncoder para P_emaildomain
cat_vars_binary = ['P_emaildomain'] + col_selection_feature_catg_num
X = binary_encode(X, cat_vars_binary)




In [76]:
X.head(2)

,D2,V128,V134,V306,V307,V308,V313,V315,V317,V318,...,V320_0,V320_1,V320_2,V320_3,V320_4,V320_5,V320_6,V320_7,V320_8,V320_9
518262,36.0,388.0,97.0,214.0,659.00,388.0,0.0,0.0,194.0,97.0,...,0,0,0,0,0,0,0,0,0,1
423813,224.0,45.0,0.0,45.0,472.75,45.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0


## Normalización y estandarización de las variables.

- Basados en el anális EDA la columna 'TransactionDT' tiene una distribución casi uniforme por lo tanto la normalización Min-Max ajusta los valores de una característica a un rango específico, típicamente entre 0 y 1. Es útil cuando deseas mantener la distribución de los datos pero ajustarla a una escala uniforme.

In [77]:
X[col_selection_feature_num].nunique()

V313     687
V134     585
V318     714
V315    1009
V306     890
D2       638
V317    1049
V128    1470
V308    1821
V307    3865
dtype: int64

In [79]:
# Aplicar MinMaxScaler a las columnas específicas
# scaler_min_max = MinMaxScaler()
# X[['TransactionDT', 'card1', 'card2', 'card3', 'card5']] = scaler_min_max.fit_transform(X[['TransactionDT', 'card1', 'card2', 'card3', 'card5']])


# Aplicar RobustScaler a 'TransactionAmt'
scaler_robust = RobustScaler()
X[col_selection_feature_num] = scaler_robust.fit_transform(X[col_selection_feature_num])




In [ ]:
X.head()

In [ ]:
# Guardar X y y en archivos CSV

dir_data_processed = '../data/processed'
ruta_archivo_X = os.path.join(dir_data_processed, 'datos_procesados.csv')
X.to_csv(ruta_archivo_X, index=False, encoding='utf-8')

ruta_archivo_y = os.path.join(dir_data_processed, 'target.csv')
y.to_csv(ruta_archivo_y, index=False, encoding='utf-8')



## Conclusión

En este proyecto de Data Science, el proceso de limpieza y preprocesamiento de datos ha sido fundamental para preparar nuestros datos antes de aplicar modelos de aprendizaje automático. A continuación, se detallan las principales etapas y decisiones tomadas durante este proceso:

1. **Selección Estratégica del Dataset**: Se cargó el dataset completo y se seleccionó estratégicamente una porción del 10%, utilizando muestreo estratificado para asegurar representatividad y evitar sesgos en nuestros modelos.

2. **Selección de Características Relevantes**: Se llevó a cabo la selección de las columnas más relevantes para nuestro análisis y modelos, descartando aquellas que no contribuían significativamente a la predicción del target.

3. **Manejo de Valores Nulos**: Se eliminaron aquellas columnas que contenían un alto porcentaje (mayor al 80%) de valores nulos, así como las filas con un porcentaje bajo de valores nulos para garantizar la integridad de los datos restantes.

4. **Separación de Variables**: Se realizó una clara separación entre las variables características (`X`) y el target (`y`), asegurando que estuvieran correctamente definidas para el entrenamiento de los modelos.

5. **Ingeniería de Características**: Basados en un análisis exploratorio de datos (EDA), se reemplazó la columna `TransactionAmt` por `TransactionAmt_Range`, una variable categórica que agrupa los valores en rangos como "muy bajo", "bajo", "medio", "alto" y "muy alto". Esta transformación facilita el manejo de variables con amplios rangos de valores, evitando posibles complicaciones durante el entrenamiento de los modelos.

6. **Combinación de Variables**: Se combinaron las columnas `addr1` y `addr2` en una sola columna (`addr_combined`), reduciendo así la dimensionalidad del dataset sin perder información relevante.

7. **Codificación de Variables Categóricas**: Se aplicaron técnicas adecuadas de codificación a las variables categóricas según su naturaleza, como codificación one-hot y binary encoding, para prepararlas para su uso en los modelos de aprendizaje automático.

8. **Normalización de Variables Numéricas**: Se normalizaron las variables numéricas para asegurar que todas estuvieran en la misma escala, lo cual es crucial para modelos que se basan en la distancia o magnitud de los atributos.

En resumen, el proceso de limpieza y preprocesamiento de datos realizado ha permitido transformar el dataset inicial en un formato apto y optimizado para la construcción de modelos predictivos. Estas etapas son fundamentales para asegurar la calidad de los resultados obtenidos y facilitar la interpretación y aplicación de los modelos en la práctica.
